In [ ]:
# Import TensorFlow library
import tensorflow as tf


In [ ]:
# Download rock paper scissors dataset
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip
  

In [ ]:
# Import library untuk mengolah dataset
import zipfile
import os

# Unzip dataset dari zip
zip_location = '/tmp/rockpaperscissors.zip'
zip_extract = zipfile.ZipFile(zip_location, 'r')
zip_extract.extractall('/tmp')
zip_extract.close()

# Utilitas untuk membersihkan folder rockpaperscissors/rps-cv-images
os.remove('/tmp/rockpaperscissors/rps-cv-images/README_rpc-cv-images.txt')
base_dir = ('/tmp/rockpaperscissors/rps-cv-images/')


In [ ]:
# Import ImageDataGenerator untuk augmentasi gambar
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Mengatur parameter untuk augmentasi gambar
# Membagi dataset ke train dan validation set sebesar 60/40
base_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.5, 1.0),
    shear_range=0.2,
    fill_mode='nearest',
    validation_split=0.4)

# Generator untuk training
train_generator = base_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

# Generator untuk validation
valid_generator = base_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
# Deklarasi class callback untuk stop training 
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.96 and logs.get('val_accuracy') > 0.96):
            print("\nAccuracy is Optimal")
            self.model.stop_training = True


In [ ]:
# Membuat model dengan sequential, convolution, pooling, dropout dan dense
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')
])


In [ ]:
# Membuat instansi callback untuk stop training
callbacks = myCallback()

# Compile model dengan loss dan optimizer
model.compile(
    loss='categorical_crossentropy',
    optimizer='Adamax',
    metrics=['accuracy'])

# Melihat summary dari model
model.summary()


In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=20,
    epochs=50,
    validation_data=valid_generator,
    validation_steps=10,
    verbose=1,
    callbacks=[callbacks]
)


In [ ]:
# Code untuk keperluan prediksi model
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
# Percobaan
uploaded = files.upload()

for fn in uploaded.keys():
    path = fn
    img = image.load_img(path, target_size=(128, 128))
    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)

if classes[0][0] > classes[0][1]:
    if classes[0][0] > classes[0][2]:
        print('Paper')
    else:
        print('Scissors')
else:
    if classes[0][1] > classes[0][2]:
        print('Rock')
    else:
        print('Scissors')
